# Concept Extraction and Topic Modeling

In [1]:
import pandas as pd
import re
import gensim
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import gensim.corpora as corpora
from pprint import pprint

## Text Preprocessing using NIPS papers dataset

In [2]:
papers = pd.read_csv('./data/NIPSPapers/papers.csv')
papers = papers.drop(columns=['id', 'event_type', 'pdf_name'])
papers['text'] = papers['paper_text'].map(lambda x: re.sub('[,\.!?]', '', x))
papers['text'] = papers['text'].map(lambda x: x.lower())
papers.head()

,year,title,abstract,paper_text,text
0,1987,Self-Organization of Associative Database and ...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,767\n\nself-organization of associative databa...
1,1987,A Mean Field Theory of Layer IV of Visual Cort...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...,683\n\na mean field theory of layer iv of visu...
2,1988,Storing Covariance by the Associative Long-Ter...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,394\n\nstoring covariance by the associative\n...
3,1994,Bayesian Query Construction for Neural Network...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...,bayesian query construction for neural\nnetwor...
4,1994,"Neural Network Ensembles, Cross Validation, an...",Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a...",neural network ensembles cross\nvalidation and...


In [3]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

data = papers.text.values.tolist()
data_words = list(sent_to_words(data))
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['self', 'organization', 'associative', 'database', 'applications', 'hisashi', 'suzuki', 'suguru', 'arimoto', 'osaka', 'university', 'toyonaka', 'osaka', 'japan', 'abstract', 'efficient', 'method', 'self', 'organizing', 'associative', 'databases', 'proposed', 'together', 'applications', 'robot', 'eyesight', 'systems', 'proposed', 'databases', 'associate']


In [4]:
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 6), (6, 1), (7, 1), (8, 3), (9, 1), (10, 2), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 4), (18, 8), (19, 1), (20, 1), (21, 2), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


## Latent Dirichlet Allocation (LDA)

In [12]:
num_topics = 4
lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word, num_topics=num_topics)
doc_lda = lda_model[corpus]

/Users/rhythmsyed/miniconda3/envs/entitylink/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
pprint(lda_model.print_topics())

[(0,
  '0.007*"learning" + 0.007*"data" + 0.006*"model" + 0.005*"function" + '
  '0.005*"algorithm" + 0.004*"set" + 0.004*"problem" + 0.004*"using" + '
  '0.003*"distribution" + 0.003*"one"'),
 (1,
  '0.005*"model" + 0.005*"algorithm" + 0.005*"data" + 0.005*"set" + '
  '0.005*"using" + 0.004*"learning" + 0.004*"number" + 0.004*"one" + '
  '0.004*"two" + 0.003*"also"'),
 (2,
  '0.006*"model" + 0.006*"learning" + 0.005*"function" + 0.005*"algorithm" + '
  '0.005*"data" + 0.004*"set" + 0.004*"time" + 0.004*"one" + 0.004*"figure" + '
  '0.004*"using"'),
 (3,
  '0.007*"model" + 0.007*"learning" + 0.005*"data" + 0.005*"time" + '
  '0.005*"set" + 0.004*"using" + 0.004*"algorithm" + 0.004*"neural" + '
  '0.004*"two" + 0.003*"figure"')]


/Users/rhythmsyed/miniconda3/envs/entitylink/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
import os

/Users/rhythmsyed/miniconda3/envs/entitylink/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./data/LDA_results/lda_' + str(num_topics))

/Users/rhythmsyed/miniconda3/envs/entitylink/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
with open(LDAvis_data_filepath, 'wb') as f:
    pickle.dump(LDAvis_prepared, f)

/Users/rhythmsyed/miniconda3/envs/entitylink/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
    
pyLDAvis.save_html(LDAvis_prepared, './data/LDA_results/lda_' + str(num_topics) +'.html')

/Users/rhythmsyed/miniconda3/envs/entitylink/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
LDAvis_prepared

/Users/rhythmsyed/miniconda3/envs/entitylink/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.005064  0.004031       1        1  27.846862
3      0.006271  0.002382       2        1  27.566202
1     -0.001122 -0.004754       3        1  24.546525
2     -0.000085 -0.001659       4        1  20.040411, topic_info=             Term          Freq         Total Category  logprob  loglift
471        method  31786.000000  31786.000000  Default  30.0000  30.0000
317      function  60703.000000  60703.000000  Default  29.0000  29.0000
794          time  53261.000000  53261.000000  Default  28.0000  28.0000
2733  probability  29622.000000  29622.000000  Default  27.0000  27.0000
870            xi  26947.000000  26947.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
559   performance   6069.993914  27920.478417   Topic4  -6.2135   0.0814
515        number   7958.453970  46875.796924   Topic4  -5.9426  -0.1658
1115       models   7528.527161  42666.838274   Topic4  -5.9982  -0.1273
325         given   7327.695549  40388.279970   Topic4  -6.0252  -0.0995
1212      results   7235.300500  40392.023669   Topic4  -6.0379  -0.1122

[402 rows x 6 columns], token_table=        Topic      Freq    Term
term                           
48439       1  0.145229   aaaaa
48439       2  0.290457   aaaaa
48439       3  0.145229   aaaaa
48439       4  0.508300   aaaaa
1303        1  0.218908  action
...       ...       ...     ...
60822       4  0.138469     zxy
168742      1  0.633806    zxyi
168742      2  0.126761    zxyi
168742      3  0.126761    zxyi
168742      4  0.126761    zxyi

[1142 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 3])

## Hierarchical Dirichlet Process

In [12]:
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models import HdpModel

/Users/rhythmsyed/miniconda3/envs/entitylink/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
hdp = gensim.models.HdpModel(corpus=corpus, id2word=id2word)

/Users/rhythmsyed/miniconda3/envs/entitylink/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
hdp.print_topics(num_topics=5, num_words=10)

/Users/rhythmsyed/miniconda3/envs/entitylink/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.006*model + 0.006*learning + 0.006*data + 0.005*algorithm + 0.004*set + 0.004*function + 0.004*using + 0.004*one + 0.003*time + 0.003*two'),
 (1,
  '0.006*learning + 0.006*model + 0.005*data + 0.005*algorithm + 0.004*one + 0.004*function + 0.004*set + 0.004*using + 0.004*time + 0.003*training'),
 (2,
  '0.006*learning + 0.005*network + 0.005*data + 0.005*figure + 0.004*model + 0.004*time + 0.004*set + 0.004*one + 0.004*neural + 0.004*algorithm'),
 (3,
  '0.005*learning + 0.005*model + 0.004*data + 0.004*figure + 0.004*using + 0.004*function + 0.004*one + 0.004*set + 0.004*network + 0.004*time'),
 (4,
  '0.005*model + 0.004*figure + 0.004*input + 0.004*neural + 0.004*learning + 0.003*networks + 0.003*data + 0.003*time + 0.003*wavelet + 0.003*set')]

In [23]:
# Visualize the topics
LDAvis_data_filepath = os.path.join('./data/LDA_results/hlda_' + str(num_topics))

/Users/rhythmsyed/miniconda3/envs/entitylink/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
LDAvis_prepared = pyLDAvis.gensim.prepare(hdp, corpus, id2word)
with open(LDAvis_data_filepath, 'wb') as f:
    pickle.dump(LDAvis_prepared, f)

/Users/rhythmsyed/miniconda3/envs/entitylink/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/rhythmsyed/Desktop/GTRI/entitylink/KnowledgeGraph_Builder/src/pyldavis/pyLDAvis/gensim.py:51: RuntimeWarning: invalid value encountered in true_divide
  doc_topic_dists = gamma / gamma.sum(axis=1)[:, None]


ValidationError: 
 * Not all rows (distributions) in doc_topic_dists sum to 1.

## Hierarchical Clustering: Taxogen